In [1]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import statsmodels.formula.api as sm
from sklearn.metrics import mean_squared_error,r2_score,explained_variance_score
from datetime import date
import matplotlib.pyplot as plt
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="p0o9i8u7",
  database="jockey"
)

#file0 = "/Users/apple/Workspace/intelliJ/jockey/regression_20190127.csv"

#dataset0 = pd.read_csv(file0)

#'horse_winCount','jockey_winCount',,'declaredHorseWeight','addedWeight'
#
#features=['draw','horse_winPer','jockey_winPer','horse_newDistance','horse_newHorse',
#         'horse_last4SpeedRate','horse_latestSpeedRate','Days_from_lastRace','weightRadio','weightRD',
#           'horseFx','horseFx_Distance','jockeyFx','jockeyFx_Distance','propByWinOdds']
# 'horse_last4SpeedRate','horse_latestSpeedRate'       
features=['drawFx','horse_newDistance','horse_newHorse',
         'Days_from_lastRace','weightRD',
           'horseFx','horseFx_Distance','jockeyFx','jockeyFx_Distance','propByWinOdds']

target_fea='place_edit'
target=[target_fea]
sql = 'select raceDate,raceSeqOfDay,distance,raceClass, horseId, place,lbs,winOdds,'
for aFeature in features:
    sql = sql + aFeature +", "
#sql = sql+ target_fea+ " from racecard"
sql = sql+ " (case when place =1 then 1 when (place <7 & place >1) then 2 else 3 end) as place_edit from racecard"
#where racedate='20190213'"

columns=['raceDate','raceSeqOfDay','distance','raceClass',*features,*target]

dataset0 = pd.read_sql(sql,mydb)
#print(dataset0)
#allDates =['20190217']
allDates= dataset0.raceDate.unique()
allDates.sort()
tableName='racecard'


from time import gmtime, strftime
updatedate = strftime("%Y%m%d%H%M%S", gmtime())

In [4]:
count=0
for raceDate in allDates:
    if (raceDate <= '20190211'):
        continue
    print(raceDate)
    dataset_ofADay = dataset0[(dataset0['raceDate']==raceDate)]
    
    allSeqs = dataset_ofADay.raceSeqOfDay.unique()

    for seq in allSeqs:
        dataset_test =  dataset_ofADay[(dataset_ofADay['raceSeqOfDay']==seq)]
        distance = dataset_test['distance'].iloc[0]
        raceClass = dataset_test['raceClass'].iloc[0]
        
        #print(raceDate,seq,distance,raceClass)
        count =count+1
        dataset_train = dataset0[(dataset0['raceClass']==raceClass) & (dataset0['distance']==distance) 
                         & (dataset0['raceDate']!=raceDate)]
        #dataset_train.shape
        if(dataset_train.size==0):
            continue
        #regression model of lbw or finishTime
        X_train=dataset_train[features]
        y_train=dataset_train[target] #lbw

        X_test=dataset_test[features]
        y_test=dataset_test[target] #lbw


        reg = LinearRegression()
        reg.fit(X_train,y_train)
        y_test_predicted=reg.predict(X_test)
        
        #plt.scatter(y_test, y_test_predicted,  color='black')
        #plt.show()
        #print("Mean squared error: %.2f"
        #        % mean_squared_error(y_test, y_test_predicted))
        # Explained variance score: 1 is perfect prediction
        # print('Variance score: %.2f' % r2_score(y_test, y_test_predicted))
        #print('explained_variance_score score: %.2f' % explained_variance_score(y_test, y_test_predicted))
        #explained_variance_score(y_test, y_test_predicted)
        df = pd.DataFrame(data=y_test_predicted, index=y_test.index,columns=['finishTime_predicted'])
        dataset_test_toshow=dataset_test[['raceDate','raceSeqOfDay','distance','raceClass','horseId',
                                          'place','lbs',target_fea]]
        dataset_test_withPredicted=dataset_test_toshow.merge(df,how='inner',left_index=True,right_index=True)
        dataset_test_withPredicted = dataset_test_withPredicted.sort_values(['finishTime_predicted'])
        #print(dataset_test_withPredicted)
       
        predicted_place=1
        for index, row in dataset_test_withPredicted.iterrows():
            mycursor = mydb.cursor()
            raceDate_str = str(row['raceDate'])
            raceSeqOfDate =str(row['raceSeqOfDay'])
            horseId = str(row['horseId'])
            finishTime_predicted =str(row['finishTime_predicted'])
            ##print("Andrea:",raceDate_str,raceSeqOfDate,horseId,finishTime_predicted)
            sql = ("UPDATE "+tableName+ " SET predicted_date=" + updatedate +", predicted_place="+ str(predicted_place) +", predicted_finishTime = "+ finishTime_predicted +
            " WHERE raceDate = "+ raceDate_str + " and raceSeqOfDay = "+raceSeqOfDate + " and horseId ='"+horseId+ "' ")
            
            #print(predicted_place)
            predicted_place =predicted_place+1
            #print(sql)
            mycursor.execute(sql)
            mydb.commit()
            #print(mycursor.rowcount, " record(s) updated" ,row['raceSeqOfDay'] ,",", row['horseId'] )

    print("total record:", count)

20190213


/Users/apple/miniconda3/lib/python3.6/site-packages/sklearn/linear_model/base.py:485: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  linalg.lstsq(X, y)


total record: 9
20190217
total record: 19
20190220
total record: 27
20190224
total record: 37
20190227
total record: 45
20190302
total record: 55
20190306
total record: 63
20190310
total record: 74
20190313
total record: 82
20190317
total record: 92
20190320
total record: 100
20190324
total record: 109
20190327
total record: 117
20190331
total record: 127
20190403
total record: 135
20190407
total record: 145
20190410
total record: 153
